# TALLER 10 - API TWITTER

In [1]:
##pip install tweepy
import tweepy
import pandas as pd

In [2]:
# Leer llaves
keys = pd.read_csv("keys_demo.csv", header=None)
keys = dict(zip(keys[0],keys[1]))

In [3]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_key_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
tweets = tweepy.Cursor(api.search,
                       q=['Navidad -RT'],
                       lang='es',
                       geocode=f'4.649178,-74.062827,52mi',
                       tweet_mode='extended',
                       include_rts=False).items(500)

In [5]:
t = []
for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogota")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    t.append(temp)

In [6]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data

,tweet,date,city,user_location,id,link
0,Contando los días para estar en casa armando e...,2020-11-07 16:05:36,Bogota,"Bogotá, Colombia",1325107141947109377,https://twitter.com/i/web/status/1325107141947...
1,Estoy tan emocionada de que llegue diciembre p...,2020-11-07 16:03:35,Bogota,"Bogotá, Colombia.",1325106635967246336,https://twitter.com/i/web/status/1325106635967...
2,Cómo sería un viaje a Cali de navidad? 🥰,2020-11-07 15:46:38,Bogota,"Bogotá, Colombia.",1325102366358708224,https://twitter.com/i/web/status/1325102366358...
3,Veamos.\n\nNo soy heterosexual\nMi familia no ...,2020-11-07 15:45:01,Bogota,Bogotá,1325101960119377920,https://twitter.com/i/web/status/1325101960119...
4,Hoy llega la navidad en esta casa. ❤️❤️❤️❤️🌲🌲🌲,2020-11-07 15:35:37,Bogota,"Bogotá , Colombia",1325099596167712770,https://twitter.com/i/web/status/1325099596167...
...,...,...,...,...,...,...
495,@MarioRobertoP El inicio de la Navidad. En ser...,2020-11-05 00:58:17,Bogota,"Bogotá, Colombia",1324154031409946624,https://twitter.com/i/web/status/1324154031409...
496,@MarioRobertoP Inicio de la navidad,2020-11-05 00:58:13,Bogota,Bogotá Colombia,1324154015182180353,https://twitter.com/i/web/status/1324154015182...
497,¿Qué trabajadores se beneficiarán de la prima ...,2020-11-05 00:53:46,Bogota,Ibagué,1324152896284446722,https://twitter.com/i/web/status/1324152896284...
498,Felicidades canal red + x mostrar navidad en p...,2020-11-05 00:47:27,Bogota,"Bogotá, D.C., Colombia",1324151307708542981,https://twitter.com/i/web/status/1324151307708...


In [7]:
data = data[['tweet','date','city']]
data.head()

,tweet,date,city
0,Contando los días para estar en casa armando e...,2020-11-07 16:05:36,Bogota
1,Estoy tan emocionada de que llegue diciembre p...,2020-11-07 16:03:35,Bogota
2,Cómo sería un viaje a Cali de navidad? 🥰,2020-11-07 15:46:38,Bogota
3,Veamos.\n\nNo soy heterosexual\nMi familia no ...,2020-11-07 15:45:01,Bogota
4,Hoy llega la navidad en esta casa. ❤️❤️❤️❤️🌲🌲🌲,2020-11-07 15:35:37,Bogota


In [8]:
import re
 
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'
def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    
    for mention in mentions:
        text = text.replace(mention,"")
    return text, links, hashtags, mentions

In [9]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data.head()

C:\Users\cikan\Anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,tweet,date,city,clean,links,hashtags,mentions
0,Contando los días para estar en casa armando e...,2020-11-07 16:05:36,Bogota,Contando los días para estar en casa armando e...,[],[],[]
1,Estoy tan emocionada de que llegue diciembre p...,2020-11-07 16:03:35,Bogota,Estoy tan emocionada de que llegue diciembre p...,[],[],[]
2,Cómo sería un viaje a Cali de navidad? 🥰,2020-11-07 15:46:38,Bogota,Cómo sería un viaje a Cali de navidad? 🥰,[],[],[]
3,Veamos.\n\nNo soy heterosexual\nMi familia no ...,2020-11-07 15:45:01,Bogota,Veamos.\n\nNo soy heterosexual\nMi familia no ...,[https://t.co/j7kBwB5lVz],[],[]
4,Hoy llega la navidad en esta casa. ❤️❤️❤️❤️🌲🌲🌲,2020-11-07 15:35:37,Bogota,Hoy llega la navidad en esta casa. ❤️❤️❤️❤️🌲🌲🌲,[],[],[]


In [10]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [11]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
1,2020-11-05,203
0,2020-11-06,223
2,2020-11-07,74


In [12]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})
layout = go.Layout(title='Número de tuits por día')
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

### Al parecer las personas estan armando navidad este fin de semana